In [1]:
from datasets.dataset import HAM_datasets
from models.meta import Meta
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import torch
import timm
from tensorboardX import SummaryWriter
import numpy as np
import pandas as pd
import os
import sys
import shutil
from PIL import Image
from models.basenet import *
from utils import *
from configs.config_setting import setting_config
from copy import deepcopy
import sklearn.metrics as metrics
from torch.cuda.amp import autocast, GradScaler
import torch.nn.init as init

import warnings
warnings.filterwarnings("ignore")

config = setting_config

/home/wangjie/anaconda3/envs/dataEngineering/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def preprocess_batch(batch):
    support_images = batch['support_images'].squeeze(0)
    support_masks = batch['support_masks'].squeeze(0)
    query_images = batch['query_images'].squeeze(0)
    query_masks = batch['query_masks'].squeeze(0)
    return support_images, support_masks, query_images, query_masks

# the function of copying the images
def copy_file_to_folder(source_file, dest_folder):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    dest_path = os.path.join(dest_folder, os.path.basename(source_file))
    shutil.copy(source_file, dest_path)

def evaluation_basenet(base_net,query_images,query_masks,criterion):
        predicted = base_net(query_images)
        np.set_printoptions(threshold=np.inf)
        loss = criterion(predicted,query_masks)
        predicted = torch.argmax(predicted,dim=1).long()
        predict_numpy = predicted.detach().cpu().numpy().reshape(-1)
        masks_numpy = query_masks.long().detach().cpu().numpy().reshape(-1)
        accuracy = metrics.accuracy_score(masks_numpy,predict_numpy)
        f1_score = metrics.f1_score(masks_numpy,predict_numpy,average=None)
        return accuracy,f1_score,loss

def initialize_weights_he(model):
    for param in model.parameters():
        init.kaiming_uniform_(param, mode='fan_in', nonlinearity='relu')

def initialize_weights_xavier(model):
    for param in model.parameters():
        init.xavier_uniform_(param)

def initialize_weights_normal(model):
    for param in model.parameters():
        init.normal_(param, mean=0, std=1)        


In [3]:
print('#----------Generating data----------#')
images_resources_path = "./data/HAM10000/origin/images/"         # the resource folder of images
masks_resources_path = "./data/HAM10000/origin/masks/"           # the resource folder of masks
ratio = 0.8     # the dataset and testset ratio
categories = config.categories
categories_dictionary = {}
category_id = 1
# prepare the csv for groundtruth
origin_groundtruth_csv = "./data/HAM10000/origin/groundtruth/HAM10000_groundtruth.csv"   # read the csv file
origin_groundtruth = pd.read_csv(origin_groundtruth_csv)    # read the csv file of groundtruth

# generating the folders for each category in train folder and test folder
# create folders for each categories
trainset_images_path = "./data/HAM10000/train/images/"     # the images path for train dataset
trainset_masks_path = "./data/HAM10000/train/masks/"     # the masks path for train dataset
testset_images_path = "./data/HAM10000/test/images/"     # the images path for test dataset
testset_masks_path = "./data/HAM10000/test/masks/"      # the masks path for test dataset

for category in categories:
    # prepare the address for folders
    category_images_train_path = os.path.join(trainset_images_path,category)
    category_masks_train_path = os.path.join(trainset_masks_path,category)
    category_images_test_path = os.path.join(testset_images_path,category)
    category_masks_test_path = os.path.join(testset_masks_path,category)
    #delete the previously exsited folders
    shutil.rmtree(category_images_train_path)
    shutil.rmtree(category_masks_train_path)
    shutil.rmtree(category_images_test_path)
    shutil.rmtree(category_masks_test_path)
    # create corresponding folder for each categories
    os.makedirs(category_images_train_path, exist_ok=True)
    os.makedirs(category_masks_train_path, exist_ok=True)
    os.makedirs(category_images_test_path, exist_ok=True)
    os.makedirs(category_masks_test_path, exist_ok=True)

    # generate the data in trainset and testset for each categories
    dest_folder_images = "./data/HAM10000/train/images/"+category    # the destination train set folder of copying the images
    dest_folder_masks = "./data/HAM10000/train/masks/"+category    # the destination trian set folder of copying the masks
    dest_folder_images_change = "./data/HAM10000/test/images/"+category     # the destination folder of test set images
    dest_folder_masks_change = "./data/HAM10000/test/masks/"+category      # the destination folder of test set masks
    data_categories = origin_groundtruth[origin_groundtruth['dx'] == category]      # extract each categories 
    length_categories = len(data_categories)
    chaneg_folder_point = math.floor(length_categories * ratio)     # get the point to change directory name 
    elements_count = 0
    for image_name in data_categories['image_id']:      # each image_id in each categories
        if elements_count == chaneg_folder_point:
            dest_folder_images = dest_folder_images_change
            dest_folder_masks = dest_folder_masks_change
        images_file = image_name+".jpg"
        masks_file = image_name+"_segmentation.png"
        source_image = images_resources_path+images_file    # the full path of source of image : path + image file name
        source_mask = masks_resources_path+masks_file       # the full path of source of mask : path + mask file name
        copy_file_to_folder(source_image,dest_folder_images)
        # masks should be preprocess to the form of output for network (Width*Height*Category)
        image = Image.open(source_mask)
        image_array = np.array(image)
        image_array[image_array == 255] = category_id
        image = Image.fromarray(image_array)
        image.save(os.path.join(dest_folder_masks, masks_file))
        elements_count +=1
    categories_dictionary[category] = category_id       # add the category id in the categories_dictionary
    category_id += 1

#----------Generating data----------#


In [4]:
print('#----------GPU init----------#')
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu_id
set_seed(config.seed)
device = torch.device('cuda')
torch.cuda.empty_cache()

#----------GPU init----------#


In [5]:
print('#----------Prepareing Model----------#')
in_channels = config.in_channels
out_channels = config.num_classes
base_net = UnetPlusPlus(in_channels,out_channels)
initialize_weights_normal(base_net)
base_net = base_net.to(device)

#----------Prepareing Model----------#


In [6]:
print('#----------Prepareing loss, opt, sch and amp----------#')
criterion = config.criterion
meta_optimizer = get_optimizer(config, base_net)
meta_scheduler = get_scheduler(config, meta_optimizer)
criterion = criterion.to(device)

#----------Prepareing loss, opt, sch and amp----------#


In [7]:
print('#----------Set other params----------#')
min_loss = 999
start_epoch = 1
min_epoch = 1

#----------Set other params----------#


In [8]:
print('#----------Start training----------#')
print(
            "batch_size:%d, %d-way, %d-shot, %d-query, %d-resizeh, %d-resizew"
            % (
                config.batch_size,
                config.n_way,
                config.k_shot,
                config.k_query,
                config.resize_h,
                config.resize_w
            )
        )
torch.cuda.empty_cache()
for epoch in range(start_epoch, config.epoch_num+1):
    step = 0        # according to the step, decide to print the result or do the evaluation
    # create the dataset and dataloader
    train_dataset = HAM_datasets(config, train=True)
    train_loader = DataLoader(
        train_dataset, batch_size=config.dataloader_bs, num_workers=config.num_workers)
    test_dataset = HAM_datasets(config, train=False)
    test_loader = DataLoader(
        test_dataset, batch_size=config.dataloader_bs, num_workers=config.num_workers)
    # numerate to train
    for i,batch in enumerate(train_dataset):
        # claer the meta_optimizer, setting zero
        meta_optimizer.zero_grad()
        support_images, support_masks, query_images, query_masks = preprocess_batch(batch)
        support_images, support_masks, query_images, query_masks = support_images.to(device), support_masks.to(device), query_images.to(device), query_masks.to(device)
        support_masks = torch.squeeze(support_masks,dim=1).long()
        query_masks = torch.squeeze(query_masks,dim=1).long()
        # deep copy the base_net for inner loop, preparing the loss and optimizer
        temp_net = deepcopy(base_net)
        inner_optimizer = optim.Adam(temp_net.parameters(), lr=config.inner_lr)
        for inner_step in range(config.inner_steps):
            inner_optimizer.zero_grad()
            predicted = temp_net(support_images)
            loss = criterion(predicted,support_masks)
            loss.backward()
            inner_optimizer.step()
        query_predicted = temp_net(query_images)
        loss = criterion(query_predicted,query_masks)
        loss.backward()
        meta_optimizer.step()
        torch.cuda.empty_cache()
    torch.cuda.empty_cache()
    accuracy,f1_score,loss_score = evaluation_basenet(base_net,query_images,query_masks,criterion)
    log_info = f'Epoch: {epoch}, loss: {loss_score}, accuracy: {accuracy:.4f}, f1_score: {f1_score}'
    print("Evaluation:",log_info)
    torch.cuda.empty_cache()

#----------Start training----------#
batch_size:1, 3-way, 5-shot, 5-query, 128-resizeh, 128-resizew
Evaluation: Epoch: 1, loss: 4.171700772035182e+31, accuracy: 0.1127, f1_score: [0.04315239 0.09192558 0.12952847 0.20611817]
Evaluation: Epoch: 2, loss: 3.950440267107345e+31, accuracy: 0.1133, f1_score: [0.04291876 0.16756642 0.15527153 0.1339893 ]
Evaluation: Epoch: 3, loss: 4.268528960201724e+31, accuracy: 0.1090, f1_score: [0.04421623 0.08843112 0.18301145 0.14360972]
Evaluation: Epoch: 4, loss: 4.221965006192791e+31, accuracy: 0.1065, f1_score: [0.04352051 0.1152186  0.17806737 0.12525463]
Evaluation: Epoch: 5, loss: 3.9911300505587704e+31, accuracy: 0.1318, f1_score: [0.0439204  0.10793336 0.22138649 0.16760767]
Evaluation: Epoch: 6, loss: 4.093725056670038e+31, accuracy: 0.1128, f1_score: [0.0441927  0.10871493 0.12816848 0.1980218 ]
Evaluation: Epoch: 7, loss: 3.8919550955911925e+31, accuracy: 0.1186, f1_score: [0.04102264 0.14728115 0.17561586 0.15210372]
Evaluation: Epoch: 8, l